In [ ]:
import pysam
import pandas as pd
import json
import re
from typing import List, Any

ModuleNotFoundError: No module named 'Levenshtein'

In [18]:

## original bases and motifs (pre sequencing)

with open(r"/mnt/c/Users/Parv/Doc/HelixWorks/Basecalling/code/motifcaller/data/synthetic/synthetic_dataset_preprocess_runs/25.2.25/info.json", 'r') as f:
    info_dict = json.load(f)

read_ids = info_dict['read_ids']

In [33]:
motif_picks = info_dict['motif_choices']
motif_labels = info_dict['motif_labels']
motif_links = info_dict['link_choices']

In [68]:
motif_picks

['CCACTAACGTAGAGTACTGC',
 'GTCGAGGATAATTGTGCAGA',
 'CACCCCCGTATTTTGAGCGG',
 'ATCCTAGCGTTTTCGAACGT',
 'CACGTTGTTTTATCGTCGTC',
 'GTCTCCCCAGGAACGGTGCC',
 'GCATGAAGACACTAGCGCCT',
 'CAGCGCCACCAACTCCACAA',
 'GTGCGGCCGT',
 'GGGACAGCTA',
 'ACATCAGTCG',
 'CCTTCATACC',
 'GTCCATCCAC',
 'ATGACGTCGG',
 'AGCTTTGAGC',
 'TTCACACCTT']

In [80]:
filepath = r"calls.bam"
samfile = pysam.AlignmentFile(filepath, "rb", check_sq=False)
seq_id = []
positions = []


def get_read_id(split_x: List[Any]) -> str:
    x_id = split_x[0]
    pattern = r">.*!"
    return re.search(r">.*!", x_id)[0][1:-10].replace('!', '')

def get_base_sequence(split_x: List[Any]) -> str:
    return split_x[9].strip()

def create_base_seq_from_motifs(
        motif_seq: List[List[int]], motif_picks:List[str],
        motif_links: List[str]) -> str:
    
    return "".join([
        motif_picks[i-1] for i in motif_seq])


def motif_search(base_seq: str, motif_picks: List[str], motif_links: List[str]):

    motif_length = len(motif_picks[0]) - 13
    motif_prediction = []
    # Sliding window searching for motif picks
    for i in range(0, len(base_seq) - motif_length, motif_length):
        subseq = base_seq[i: i + motif_length]
        for ind, j in enumerate(motif_picks):
            if subseq in j:
                motif_prediction.append(ind+1)

    return motif_prediction


iter = samfile.fetch(until_eof=True)
for ind, x in enumerate(iter):
    x = str(x)
    split_x = x.split()
    strand_orientation = split_x[0][-1]
    read_id = get_read_id(split_x)
    base_seq_prediction = get_base_sequence(split_x)

    motif_label = motif_labels[read_ids.index(read_id)]
    base_seq_true = create_base_seq_from_motifs(
        motif_seq=motif_label, motif_picks=motif_picks, motif_links=motif_links)
    
    motif_prediction = motif_search(
        base_seq=base_seq_prediction, motif_picks=motif_picks, motif_links=motif_links)
    print(motif_prediction)
    print(motif_label)
    print(strand_orientation)
    print()
    
    




    if ind == 10:
        break

[14, 8, 14, 14, 15, 5, 5, 15, 15, 8, 8, 16, 1, 16, 2, 2, 16, 5, 5, 16, 8, 8]
[9, 1, 9, 9, 4, 9, 9, 6, 9, 9, 8, 9, 10, 1, 10, 10, 3, 10, 10, 5, 10, 10, 6, 10, 11, 4, 11, 11, 6, 11, 11, 7, 11, 11, 8, 11, 12, 3, 12, 12, 6, 12, 12, 7, 12, 12, 8, 12, 13, 2, 13, 13, 4, 13, 13, 7, 13, 13, 8, 13, 14, 1, 14, 14, 2, 14, 14, 6, 14, 14, 8, 14, 15, 2, 15, 15, 5, 15, 15, 7, 15, 15, 8, 15, 16, 1, 16, 16, 2, 16, 16, 5, 16, 16, 8, 16]
+

[3, 10, 8, 11, 2, 2, 11, 4, 4, 11, 5, 11, 7, 7, 2, 3, 6, 6, 12, 7, 7, 13, 1, 1, 13, 13, 5, 13, 6, 6, 13, 2, 2, 14, 5, 5, 14, 6, 6, 14, 8, 8, 3, 15, 4, 1, 15, 6, 6, 15, 7, 7, 3, 15, 8, 8, 1, 16, 2, 16, 4, 4, 16, 5, 5]
[9, 1, 9, 9, 2, 9, 9, 4, 9, 9, 7, 9, 10, 1, 10, 10, 2, 10, 10, 3, 10, 10, 7, 10, 11, 2, 11, 11, 4, 11, 11, 5, 11, 11, 7, 11, 12, 2, 12, 12, 3, 12, 12, 6, 12, 12, 7, 12, 13, 1, 13, 13, 4, 13, 13, 5, 13, 13, 6, 13, 14, 2, 14, 14, 5, 14, 14, 6, 14, 14, 8, 14, 15, 4, 15, 15, 6, 15, 15, 7, 15, 15, 8, 15, 16, 1, 16, 16, 2, 16, 16, 4, 16, 16, 5, 16]
+

[12, 3, 3,

In [60]:
t = [1,2,3]
s = [4,5]
t + s

[1, 2, 3, 4, 5]

In [55]:

# Squiggle database (that needs added motif search predictions)
dataset_path = r"/mnt/c/Users/Parv/Doc/HelixWorks/Basecalling/code/motifcaller/data/synthetic/pickled_datasets/25_2_25.pkl"
dataset = pd.read_pickle(dataset_path)


ModuleNotFoundError: No module named 'numpy._core.numeric'